In [1]:
# DataFrame
import pandas as pd

# nltk
import nltk
from nltk.corpus import stopwords
from  nltk.stem import SnowballStemmer

# Utility
import re
import numpy as np
import os
from collections import Counter
import logging
import time
import pickle
import itertools

nltk.download('stopwords')

# DATASET
DATASET_COLUMNS = ["target", "ids", "date", "flag", "user", "text"]
DATASET_ENCODING = "ISO-8859-1"

# TEXT CLENAING
TEXT_CLEANING_RE = "@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+"

dataset_path = os.path.join("input","training.1600000.processed.noemoticon.csv")
print("Open file:", dataset_path)
df = pd.read_csv(dataset_path, encoding =DATASET_ENCODING , names=DATASET_COLUMNS)

stop_words = stopwords.words("english")
stemmer = SnowballStemmer("english")

def preprocess(text, stem=False):
    # Remove link,user and special characters
    text = re.sub(TEXT_CLEANING_RE, ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
        if token not in stop_words:
            if stem:
                tokens.append(stemmer.stem(token))
            else:
                tokens.append(token)
    return " ".join(tokens)

df.text = df.text.apply(lambda x: preprocess(x))

texts = []
labels = []
texts_test = []
labels_test = []
for i in range (45000):
    texts.append(df.text[i+5000])
    labels.append(0)
for i in range (45000):
    texts.append(df.text[i+805000])
    labels.append(1)
for i in range (5000):
    texts_test.append(df.text[i])
    labels_test.append(0)
for i in range (5000):
    texts_test.append(df.text[i+800000])
    labels_test.append(1)
    
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 35  # We will cut reviews after 100 words
max_words = 10000  # We will only consider the top 10,000 words in the dataset

tokenizer = Tokenizer(num_words=max_words)  # https://keras.io/preprocessing/text/
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)  # This turns strings into lists of integer indices.

d = 0
for i in range (len(sequences)):
    if len(sequences[i-d]) == 0:
        del sequences[i-d]
        del labels[i-d]
        d = d+1   

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)

# Split the data into a training set and a validation set
# But first, shuffle the data, since we started from data
# where sample are ordered (all negative first, then all positive).
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

onehot = np.zeros((data.shape[0], max_words))
for i in range(data.shape[0]):
    for j in range(maxlen):
        temp2 = data [i]
        if temp2[j] != 0:
            a = temp2[j]
            onehot[i][a] = 1
            
from sklearn.model_selection import train_test_split
from sklearn.metrics.classification import accuracy_score
from sklearn.metrics import confusion_matrix
from models import SupervisedDBNClassification

# Loading dataset
X, Y = onehot, labels

# Data scaling
X = (X).astype(np.float32)

# Splitting data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=2/9, random_state=0)
#X_train = X[1000:5000]
#X_test = X[0:1000]
#Y_train = Y[1000:5000]
#Y_test= Y[0:1000]
print (X_train.shape)
print (X_test.shape)
print (Y_train.shape)
print (Y_test.shape)

# Training
classifier = SupervisedDBNClassification(hidden_layers_structure=[16, 16],
                                         learning_rate_rbm=0.05,
                                         learning_rate=0.1,
                                         n_epochs_rbm=10,
                                         n_iter_backprop=10,
                                         batch_size=128,
                                         activation_function='relu',
                                         dropout_p=0.1)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/liyuantan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Open file: input/training.1600000.processed.noemoticon.csv


/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Found 51459 unique tokens.
(69405, 10000)
(19830, 10000)
(69405,)
(19830,)


In [2]:
%%time
classifier.fit(X_train, Y_train)

[START] Pre-training step:
>> Epoch 1 finished 	RBM Reconstruction error 6.280188
>> Epoch 2 finished 	RBM Reconstruction error 6.276951
>> Epoch 3 finished 	RBM Reconstruction error 6.233606
>> Epoch 4 finished 	RBM Reconstruction error 6.115917
>> Epoch 5 finished 	RBM Reconstruction error 6.004140
>> Epoch 6 finished 	RBM Reconstruction error 5.896607
>> Epoch 7 finished 	RBM Reconstruction error 5.808676
>> Epoch 8 finished 	RBM Reconstruction error 5.744710
>> Epoch 9 finished 	RBM Reconstruction error 5.697812
>> Epoch 10 finished 	RBM Reconstruction error 5.679136
>> Epoch 1 finished 	RBM Reconstruction error 0.619575
>> Epoch 2 finished 	RBM Reconstruction error 0.382291
>> Epoch 3 finished 	RBM Reconstruction error 0.322452
>> Epoch 4 finished 	RBM Reconstruction error 0.301850
>> Epoch 5 finished 	RBM Reconstruction error 0.297177
>> Epoch 6 finished 	RBM Reconstruction error 0.295873
>> Epoch 7 finished 	RBM Reconstruction error 0.301043
>> Epoch 8 finished 	RBM Reconstructi

SupervisedDBNClassification()

In [3]:
# Test
sequences = tokenizer.texts_to_sequences(texts_test)

d = 0
for i in range (len(sequences)):
    if len(sequences[i-d]) == 0:
        del sequences[i-d]
        del labels_test[i-d]
        d = d+1  

x_test = pad_sequences(sequences, maxlen=maxlen)
y_test = np.asarray(labels_test)

onehot_test = np.zeros((x_test.shape[0], max_words))
for i in range(x_test.shape[0]):
    for j in range(maxlen):
        temp2 = x_test [i]
        if temp2[j] != 0:
            a = temp2[j]
            onehot_test[i][a] = 1
            
x_test = (onehot_test).astype(np.float32)

In [4]:
%%time
y_pred = classifier.predict(x_test)
print('Done.\nAccuracy: %f' % accuracy_score(y_test, y_pred))

Done.
Accuracy: 0.746792
CPU times: user 475 ms, sys: 259 ms, total: 734 ms
Wall time: 559 ms
